In [1]:
import os, re, sys, json, csv, string, gzip
import create_book_corefs
import eval_pdnc_corefs
import pandas as pd
import numpy as np

In [61]:
from collections import defaultdict, Counter

In [2]:
novels = create_book_corefs.novels

In [3]:
novel = novels[0]

In [47]:
qa = create_book_corefs.read_booknlp_df(os.path.join('pdnc_output/', novel, novel+'.quotes'))

In [48]:
qa

,quote_start,quote_end,mention_start,mention_end,mention_phrase,char_id,quote
0,7,17,19,20,Miss Bartlett,115,""" The Signora had no business to do it , """
1,22,62,19,20,Miss Bartlett,115,""" no business at all . She promised us south r..."
2,63,70,72,72,Lucy,116,""" And a Cockney , besides ! """
3,86,92,93,93,She,116,""" It might be London . """
4,179,220,315,315,she,116,""" Charlotte , do n't you feel , too , that we ..."
...,...,...,...,...,...,...,...
2064,84379,84382,84301,84301,her,115,""" Niente . """
2065,84389,84395,84333,84333,Lucy,116,""" Mean what , George ? """
2066,84399,84616,84396,84396,He,118,""" Is it this ? Is this possible ? I 'll put a ..."
2067,84617,84622,84624,84624,Lucy,116,""" It is impossible , """


In [49]:
coref = pd.read_csv('pdnc_output/' + novel + '/coref_matches.csv')

In [7]:
coref

,COREF,matches,pdncID
0,113,[],-1
1,114,"[52, 52]",52
2,115,"[6, 6]",6
3,2085,[],-1
4,2086,[],-1
...,...,...,...
2324,2080,[],-1
2325,111,[],-1
2326,3571,[],-1
2327,3573,[],-1


In [11]:
tokdf_path = os.path.join(create_book_corefs.COREF_ROOT, novel, novel+'.tokens')
tokdf = create_book_corefs.read_booknlp_df(tokdf_path)

In [12]:
tokdf.set_index('token_ID_within_document', inplace=True)

In [13]:
tokdf

,paragraph_ID,sentence_ID,token_ID_within_sentence,word,lemma,byte_onset,byte_offset,POS_tag,fine_POS_tag,dependency_relation,syntactic_head_ID,event,inQuote
token_ID_within_document,,,,,,,,,,,,,
0,0,0,0,PART,PART,0,4,PROPN,NNP,dep,18,O,False
1,0,0,1,ONE,one,5,8,NUM,CD,nummod,1,O,False
2,1,0,2,Chapter,chapter,12,19,NOUN,NN,appos,0,O,False
3,1,0,3,I,i,20,21,NOUN,NN,nummod,0,O,False
4,1,0,4,:,:,21,22,PUNCT,:,punct,0,O,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
84691,2164,5828,15,winter,winter,374649,374655,NOUN,NN,pobj,84690,O,False
84692,2164,5828,16,into,into,374656,374660,ADP,IN,prep,84686,O,False
84693,2164,5828,17,the,the,374661,374664,DET,DT,det,84694,O,False


In [14]:
tokdf.loc[7]

paragraph_ID                    2
sentence_ID                     0
token_ID_within_sentence        7
word                            "
lemma                           "
byte_onset                     38
byte_offset                    39
POS_tag                     PUNCT
fine_POS_tag                   ``
dependency_relation         punct
syntactic_head_ID               6
event                           O
inQuote                     False
Name: 7, dtype: object

In [8]:
gold_df = pd.read_csv(os.path.join('/h/vkpriya/quoteAttr/data', novel, 'quote_info.csv'))

In [9]:
gold_df

,qID,qText,qSpan,speaker,addressee,qType,refExp,menTexts,menSpans,menEnts,...,dialogueTurn,speakerType,novel,startParaID,endParaID,startChapID,endChapID,speakerID,startByte,endByte
0,Q0-0,"The Signora had no business to do it,","[39, 76]",Miss Bartlett,['Lucy'],Explicit,said Miss Bartlett,['The Signora'],"[[39, 50]]",[['Signora Bertolini']],...,0,major,ARoomWithAView,3,3,1,1,6,39,76
1,Q0-1,no business at all. She promised us south room...,"[99, 271]",Miss Bartlett,['Lucy'],Explicit,said Miss Bartlett,"['She', 'us', 'Lucy']","[[119, 122], [132, 134], [266, 270]]","[['Signora Bertolini'], ['Miss Bartlett', 'Luc...",...,0,major,ARoomWithAView,3,3,1,1,6,99,271
2,Q1-0,"And a Cockney, besides!","[275, 298]",Lucy,['Miss Bartlett'],Explicit,said Lucy,[],[],[],...,1,major,ARoomWithAView,4,4,1,1,4,275,298
3,Q1-1,It might be London.,"[378, 397]",Lucy,['Miss Bartlett'],Explicit,said Lucy,[],[],[],...,1,major,ARoomWithAView,4,4,1,1,4,378,397
4,Q2-0,"Charlotte, don't you feel, too, that we might ...","[809, 976]",Lucy,['Miss Bartlett'],Implicit,NaN,"['Charlotte', 'you', 'we']","[[809, 818], [826, 829], [846, 848]]","[['Miss Bartlett'], ['Miss Bartlett'], ['Miss ...",...,2,major,ARoomWithAView,4,4,1,1,4,809,976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1984,Q1630-0,Niente.,"[373310, 373317]",Lucy,"['George Emerson', 'The Cabman']",Implicit,NaN,[],[],[],...,1630,major,ARoomWithAView,2160,2160,20,20,4,373310,373317
1985,Q1631-0,"Mean what, George?","[373353, 373371]",Lucy,['George Emerson'],Implicit,NaN,['George'],"[[373364, 373370]]",[['George Emerson']],...,1631,major,ARoomWithAView,2162,2162,20,20,4,373353,373371
1986,Q1632-0,Is it this? Is this possible? I'll put a marve...,"[373389, 374302]",George Emerson,['Lucy'],Anaphoric,He whispered,"['your cousin', 'we', 'she', 'her', 'we', 'she...","[[373450, 373461], [373512, 373514], [373520, ...","[['Miss Bartlett'], ['George Emerson', 'Lucy']...",...,1632,minor,ARoomWithAView,2163,2163,20,20,2,373389,374302
1987,Q1633-0,"It is impossible,","[374306, 374323]",Lucy,['George Emerson'],Explicit,murmured Lucy,[],[],[],...,1633,major,ARoomWithAView,2164,2164,20,20,4,374306,374323


In [39]:
qb2qid = {}
qb2pid = {}
for _, row in gold_df.iterrows():
    sb, eb = row['startByte'], row['endByte']
    qtt = row['qText']
    asb, aeb = create_book_corefs.get_offset_bytes(qtt, sb, eb)
    for b in range(asb, aeb):
        qb2qid[b] = row['qID']
        qb2pid[b] = row['speakerID']

In [40]:
qa_qids = defaultdict(set)
qa_pids = defaultdict(set)
sbs = []
ebs = []
for _, row in qa.iterrows():
    st, et = row['quote_start'], row['quote_end']
    if tokdf.loc[st]['word'] == '"':
        st += 1
    if tokdf.loc[et]['word'] == '"':
        et -= 1
    sb, eb = tokdf.loc[st]['byte_onset'], tokdf.loc[et]['byte_offset']
    sbs.append(sb)
    ebs.append(eb)
    for b in range(sb, eb):
        if b in qb2qid:
            qa_qids[sb].add(qb2qid[b])
            qa_pids[sb].add(qb2pid[b])

In [41]:
# qa['qID_matches'] = qa_qids

In [42]:
single = 0
mult = 0
none = 0
for sb, items in qa_qids.items():
    if len(items) > 1:
        print(sb, items)
        mult += 1
    elif len(items) == 1:
        single += 1
    else:
        none += 1

59680 {'Q214-1', 'Q214-0'}
61729 {'Q217-2', 'Q217-5', 'Q217-4', 'Q217-1', 'Q217-0', 'Q217-6', 'Q217-3'}
63730 {'Q220-0', 'Q219-0'}
106021 {'Q374-1', 'Q374-3', 'Q374-2'}
144367 {'Q520-2', 'Q520-1'}
146316 {'Q542-0', 'Q543-0'}
148447 {'Q554-0', 'Q556-0', 'Q555-0'}
149401 {'Q558-3', 'Q558-1', 'Q558-2'}
150073 {'Q562-0', 'Q561-0', 'Q560-0'}
172692 {'Q652-1', 'Q652-0'}
202754 {'Q795-0', 'Q795-1', 'Q795-2'}
240189 {'Q955-0', 'Q954-0'}
288232 {'Q1203-1', 'Q1203-2'}
293240 {'Q1239-0', 'Q1238-0'}
295527 {'Q1243-0', 'Q1243-2', 'Q1243-1'}
298786 {'Q1255-1', 'Q1255-0'}
306650 {'Q1291-0', 'Q1292-0'}
314223 {'Q1306-0', 'Q1307-0'}
350482 {'Q1490-2', 'Q1490-1'}
360077 {'Q1554-0', 'Q1555-0'}


In [43]:
single, mult, none

(1938, 20, 0)

In [52]:
qa['start_byte'] = sbs
qa['end_byte'] = ebs

In [45]:
# qa[qa['start_byte']==59680]['quote'].tolist()

In [33]:
# qa[qa['qID']!="none"]

In [38]:
# gold_df[gold_df['qID'].isin({'Q214-1', 'Q214-0'})]

In [53]:
qa

,quote_start,quote_end,mention_start,mention_end,mention_phrase,char_id,quote,start_byte,end_byte
0,7,17,19,20,Miss Bartlett,115,""" The Signora had no business to do it , """,39,76
1,22,62,19,20,Miss Bartlett,115,""" no business at all . She promised us south r...",99,271
2,63,70,72,72,Lucy,116,""" And a Cockney , besides ! """,275,298
3,86,92,93,93,She,116,""" It might be London . """,378,397
4,179,220,315,315,she,116,""" Charlotte , do n't you feel , too , that we ...",809,976
...,...,...,...,...,...,...,...,...,...
2064,84379,84382,84301,84301,her,115,""" Niente . """,373310,373317
2065,84389,84395,84333,84333,Lucy,116,""" Mean what , George ? """,373353,373371
2066,84399,84616,84396,84396,He,118,""" Is it this ? Is this possible ? I 'll put a ...",373389,374302
2067,84617,84622,84624,84624,Lucy,116,""" It is impossible , """,374306,374323


In [54]:
qa['qID_matches'] = [qa_qids[x] for x in qa['start_byte']]
qa['pID_matches'] = [qa_pids[x] for x in qa['start_byte']]

In [55]:
qa

,quote_start,quote_end,mention_start,mention_end,mention_phrase,char_id,quote,start_byte,end_byte,qID_matches,pID_matches
0,7,17,19,20,Miss Bartlett,115,""" The Signora had no business to do it , """,39,76,{Q0-0},{6}
1,22,62,19,20,Miss Bartlett,115,""" no business at all . She promised us south r...",99,271,{Q0-1},{6}
2,63,70,72,72,Lucy,116,""" And a Cockney , besides ! """,275,298,{Q1-0},{4}
3,86,92,93,93,She,116,""" It might be London . """,378,397,{Q1-1},{4}
4,179,220,315,315,she,116,""" Charlotte , do n't you feel , too , that we ...",809,976,{Q2-0},{4}
...,...,...,...,...,...,...,...,...,...,...,...
2064,84379,84382,84301,84301,her,115,""" Niente . """,373310,373317,{Q1630-0},{4}
2065,84389,84395,84333,84333,Lucy,116,""" Mean what , George ? """,373353,373371,{Q1631-0},{4}
2066,84399,84616,84396,84396,He,118,""" Is it this ? Is this possible ? I 'll put a ...",373389,374302,{Q1632-0},{2}
2067,84617,84622,84624,84624,Lucy,116,""" It is impossible , """,374306,374323,{Q1633-0},{4}


In [56]:
coref2char = {}
for coref, pid in zip(coref['COREF'], coref['pdncID']):
    coref2char[coref] = pid

In [57]:
qa['corefPID'] = [coref2char[c] for c in qa['char_id']]

In [58]:
qa

,quote_start,quote_end,mention_start,mention_end,mention_phrase,char_id,quote,start_byte,end_byte,qID_matches,pID_matches,corefPID
0,7,17,19,20,Miss Bartlett,115,""" The Signora had no business to do it , """,39,76,{Q0-0},{6},6
1,22,62,19,20,Miss Bartlett,115,""" no business at all . She promised us south r...",99,271,{Q0-1},{6},6
2,63,70,72,72,Lucy,116,""" And a Cockney , besides ! """,275,298,{Q1-0},{4},4
3,86,92,93,93,She,116,""" It might be London . """,378,397,{Q1-1},{4},4
4,179,220,315,315,she,116,""" Charlotte , do n't you feel , too , that we ...",809,976,{Q2-0},{4},4
...,...,...,...,...,...,...,...,...,...,...,...,...
2064,84379,84382,84301,84301,her,115,""" Niente . """,373310,373317,{Q1630-0},{4},6
2065,84389,84395,84333,84333,Lucy,116,""" Mean what , George ? """,373353,373371,{Q1631-0},{4},4
2066,84399,84616,84396,84396,He,118,""" Is it this ? Is this possible ? I 'll put a ...",373389,374302,{Q1632-0},{2},-1
2067,84617,84622,84624,84624,Lucy,116,""" It is impossible , """,374306,374323,{Q1633-0},{4},4


In [63]:
matches = []
iden = 0
for pID, corefID in zip(qa['pID_matches'], qa['corefPID']):
    if len(list(pID)) == 1 and corefID == list(pID)[0]:
        matches.append(True)
    else:
        matches.append(False)

In [64]:
Counter(matches)

Counter({True: 895, False: 1174})

In [65]:
len(matches)

2069

In [66]:
single+mult

1958

In [72]:
import importlib

In [80]:
importlib.reload(eval_pdnc_corefs)

<module 'eval_pdnc_corefs' from '/ssd005/home/vkpriya/bookNLP/booknlp-en/booknlpen/eval_pdnc_corefs.py'>

In [81]:
rows = []
for novel in novels:
    rows.append(eval_pdnc_corefs.eval_qa(novel))

ARoomWithAView 1938 20 0 1958
AgeOfInnocence 1578 4 0 1582
AliceInWonderland 1039 4 0 1043
AnneOfGreenGables 1709 30 0 1739
DaisyMiller 719 3 0 722
Emma 1908 73 0 1981
HandfulOfDust 2625 2 0 2627
HowardsEnd 3081 12 0 3093
NightAndDay 2786 4 0 2790
NorthangerAbbey 1014 0 0 1014
Persuasion 679 1 0 680
PrideAndPrejudice 1695 6 0 1701
SenseAndSensibility 1492 20 0 1512
SignOfFour 856 14 0 870
TheAwakening 722 2 0 724
TheGambler 1045 8 0 1053
TheInvisibleMan 1175 27 0 1202
TheManWhoWasThursday 1305 13 0 1318
TheMysteriousAffairAtStyles 2157 20 0 2177
ThePictureOfDorianGray 1483 8 0 1491
TheSportOfTheGods 804 2 0 806
TheSunAlsoRises 3224 7 0 3231


In [82]:
rows[10]

['Persuasion', 682, 680, 786, 250]

In [83]:
qares = pd.DataFrame(rows, columns=['novel', 'Ann', 'Iden', 'X', 'Correct'])

In [85]:
qares

,novel,Ann,Iden,X,Correct
0,ARoomWithAView,1989,1958,2069,895
1,AgeOfInnocence,1592,1582,1912,414
2,AliceInWonderland,1048,1043,1122,838
3,AnneOfGreenGables,1779,1739,1841,395
4,DaisyMiller,725,722,749,519
5,Emma,2109,1981,2107,774
6,HandfulOfDust,2617,2627,2732,908
7,HowardsEnd,3111,3093,3304,505
8,NightAndDay,2795,2790,2901,1156
9,NorthangerAbbey,1014,1014,1072,177


In [86]:
qares['acc'] = qares['Correct']/qares['Iden']

In [88]:
qares['iden_acc'] = qares['Iden']/qares['Ann']

In [89]:
qares.agg(np.nanmean)

Ann         1625.454545
Iden        1605.181818
X           1696.454545
Correct      644.954545
acc            0.423308
iden_acc       0.987449
dtype: float64